In [154]:
import pandas as pd
import re 
import ipywidgets as widgets
from ipywidgets import interact



In [155]:
country_population = pd.DataFrame(pd.read_excel('population_data.xlsx'))
country_population_copy = country_population


In [156]:
soc_med_geographic = pd.DataFrame(pd.read_excel('SocMed_Geographic.xlsx'))


In [157]:
#cleaning data
soc_med_geographic.columns = ['social_media_platform', 
                             'United States','Indonesia','Singapore','China','India','Vietnam','Philippines','Bangladesh']
soc_med_geographic.drop(soc_med_geographic.index[0], inplace=True)

In [158]:
#Long Data
melted_soc_med = pd.melt(soc_med_geographic, id_vars='social_media_platform', value_name='number_of_users', var_name= 'countries')
melted_soc_med.fillna(0, inplace=True)

In [159]:
#cleaning numbers
melted_soc_med['number_of_users'] = melted_soc_med['number_of_users'].apply(lambda x: str(x))
melted_soc_med['number_of_users'] = melted_soc_med['number_of_users'].apply(lambda x: x.replace(',','').replace(' ','').replace('.','').replace('\u202c',''))
melted_soc_med['number_of_users'] = melted_soc_med['number_of_users'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
melted_soc_med['number_of_users'] = melted_soc_med['number_of_users'].apply(lambda x: int(x))




In [160]:
grouped_soc = melted_soc_med.groupby(['social_media_platform','countries'])['number_of_users'].sum().reset_index()

In [161]:
def view_one():
    social_media_list = ['All']+sorted(grouped_soc['social_media_platform'].unique().tolist())
    def view(social_media_platform=''):
        if social_media_platform == 'All': 
            pd.set_option('display.max_rows', 80)
            return grouped_soc.sort_values(by='number_of_users', ascending=False)
        return grouped_soc[grouped_soc['social_media_platform']==social_media_platform].sort_values(by='number_of_users',ascending=False)

    soc_med_w = widgets.Select(options=social_media_list)
    interact(view,social_media_platform=soc_med_w)


In [162]:
def view_two():
    countries_list = ['All']+sorted(grouped_soc['countries'].unique().tolist())
    def country_view(countries=''):
        if countries == 'All':
            pd.set_option('display.max_rows', 80)
            return grouped_soc.sort_values(by='number_of_users', ascending=False)
        return grouped_soc[grouped_soc['countries']==countries].sort_values(by='number_of_users',ascending=False)


    country_w = widgets.Select(options=countries_list)
    interact(country_view, countries=country_w)
        

In [163]:
#creating df with population and percentage
grouped_soc_copy = grouped_soc.copy()
grouped_soc_with_pop = grouped_soc_copy.merge(country_population, on='countries', how='left')
grouped_soc_with_pop.drop('#', axis=1, inplace=True)
grouped_soc_with_pop['percentage_of_users'] = (grouped_soc_with_pop['number_of_users']/grouped_soc_with_pop['Population']) * 100


In [164]:
def view_three():
    social_media_list = ['All']+sorted(grouped_soc_with_pop['social_media_platform'].unique().tolist())
    def view(social_media_platform=''):
        if social_media_platform == 'All': 
            pd.set_option('display.max_rows', 80)
            return grouped_soc_with_pop.sort_values(by='percentage_of_users', ascending=False)
        return grouped_soc_with_pop[grouped_soc_with_pop['social_media_platform']==social_media_platform].sort_values(by='percentage_of_users',ascending=False)

    soc_med_w = widgets.Select(options=social_media_list)
    interact(view,social_media_platform=soc_med_w)


#1: When I choose a social media network, I am able to get a view the number of users in each
country.

In [165]:
view_one()

interactive(children=(Select(description='social_media_platform', options=('All', 'Bilibili', 'Facebook', 'Ins…

#2: When I choose a country, I am able to see all the relevant social networks its people use ranked
in descending order.

In [166]:
view_two()

interactive(children=(Select(description='countries', options=('All', 'Bangladesh', 'China', 'India', 'Indones…

#3: I can view a comparison of the percentage of users based on the population for each social
media network by the population of the country.

In [167]:
view_three()

interactive(children=(Select(description='social_media_platform', options=('All', 'Bilibili', 'Facebook', 'Ins…